In [1]:
using StanModels

ProjDir = rel_path_s("..", "scripts", "12")
cd(ProjDir)

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;
first(d[[:culture, :population, :log_pop, :society]], 5)

m12_6 = "
data {
    int N;
    int N_societies;
    int total_tools[N];
    real logpop[N];
    int society[N];
}
parameters{
    real a;
    real bp;
    vector[N_societies] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[N_societies] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:N ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StanModels/scripts/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1),"N_societies" => 10,
"total_tools" => d[:total_tools], "logpop" => d[:log_pop],
"society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__  927.853562000  3.184714488 0.050354757393 0.1159372315  754.562392
accept_stat__    0.942140191  0.080755045 0.001276849370 0.0016794265 1000.000000
   stepsize__    0.048463950  0.004672777 0.000073883087 0.0007481496   39.009752
  treedepth__    5.923500000  0.417961030 0.006608544145 0.0112121157 1000.000000
 n_leapfrog__   73.252000000 32.102618905 0.507586972975 0.8131511355 1000.000000
  divergent__    0.000000000  0.000000000 0.000000000000 0.0000000000         NaN
     energy__ -921.286832250  4.107998984 0.064953167075 0.1429295316  826.069741
            a    1.121403146  0.752510513 0.011898235915 0.0235459975 1000.000000
           bp    0.258884482  0.081372748 0.001286616116 0.0025943695  983.771679
  a_society.1   -0.216080266  0.250700621 0.003963924862 0.00603

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*